In [1]:
import numpy as np
import pandas as pd
import tensorflow
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,TimeDistributed,Embedding,Bidirectional
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from string import digits
import nltk
import re
import string

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', -1)

In [2]:
import warnings

# Filter out FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Your code here
# ...

# To reset the warning filter and display warnings again (if needed)
# warnings.resetwarnings()


In [8]:
import pandas as pd
import os

directory_path = r"C:\Users\Najeeb\Desktop\SIT Semesters\Sem 5\LAWWWWWWWWWWWWWWWWW\Dataset\nv3\nv3"

# Function to extract text between two patterns
def extract_text_between_patterns(text, start_pattern, end_pattern):
    start_index = text.find(start_pattern)
    end_index = text.find(end_pattern)
    if start_index != -1 and end_index != -1:
        return text[start_index + len(start_pattern):end_index]
    else:
        return None

# Create an empty DataFrame
data = {
    'Case Name': [],
    'Committee Members': [],
    'PROCEDURE': [],
    'THE FACTS': [],
    'THE LAW': [],
    'Judgment': []
}

df = pd.DataFrame(data)

# Loop through text files
for i in range(1, 561):  # Assuming you have files named 1.txt to 560.txt
    filename = f"{i}.txt"
    file_path = os.path.join(directory_path, filename)

    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    case_name = extract_text_between_patterns(text, "In the case of ", ",")
    committee_members = extract_text_between_patterns(text, ":", "Having deliberated")
    procedure_info = extract_text_between_patterns(text, "PROCEDURE", "THE FACTS")
    facts_info = extract_text_between_patterns(text, "THE FACTS", "THE LAW")
    law_info = extract_text_between_patterns(text, "THE LAW", "FOR THESE REASONS")
    judgment_info = extract_text_between_patterns(text, "FOR THESE REASONS", "Done in")

    # Append data to the DataFrame
    df = df.append({
        'Case Name': case_name,
        'Committee Members': committee_members,
        'PROCEDURE': procedure_info,
        'THE FACTS': facts_info,
        'THE LAW': law_info,
        'Judgment': judgment_info
    }, ignore_index=True)



In [15]:
import pandas as pd
import re
from num2words import num2words  # You may need to install the 'num2words' library


# Function to replace numbers with words
def replace_numbers_with_words(text):
    def replace(match):
        number = int(match.group(0))
        return num2words(number)

    return re.sub(r'\d+', replace, text)

df['THE FACTS'] = df['THE FACTS'].astype(str)
df['THE LAW'] = df['THE LAW'].astype(str)
df['Judgment'] = df['Judgment'].astype(str)
# Apply the function to the DataFrame column
df['THE FACTS'] = df['THE FACTS'].apply(replace_numbers_with_words)
df['THE LAW'] = df['THE LAW'].apply(replace_numbers_with_words)
df['Judgment'] = df['Judgment'].apply(replace_numbers_with_words)


def remove_symbols(text):
    return re.sub(r'[^\w\s]', '', text)

# Apply the function to the specified columns
df['THE FACTS'] = df['THE FACTS'].apply(remove_symbols)
df['THE LAW'] = df['THE LAW'].apply(remove_symbols)
df['Judgment'] = df['Judgment'].apply(remove_symbols)


In [16]:
# # df = df[:1]
pd.set_option('display.max_colwidth', -1)
df.head(1)

Case Name  \
0  Terge v. Hungary   

                                                                                                        Committee Members  \
0  \n\nFaris Vehabović, President,Carlo Ranzoni,Péter Paczolay, judges,and Andrea Tamietti, Deputy Section Registrar,\n\n   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      PROCEDURE  \
0  \n\n1.  The case originated in an application (no. 3625/15) against Hungary lodged with the Court under Article 34 of the Convention for the Protection of Human Rights and Fundamental Freedoms (“the Convention”) by a Hungarian national, Mr István Terge (“the applicant”), on 9 January 2015.\n\n2.  The applicant was represented by Mr D. Karsai, a lawyer practising in Budapest. The Hungarian Government (“the Government”) were represented by Mr Z. Tallódi, Agent at the Ministry of Justice.\n\n3.  On 19 January 2017 the application was communicated to the Government.\n\n   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [17]:
df.shape

(560, 6)

In [18]:
df["Committee Members"].isnull().sum()

11

In [19]:
df["Judgment"].isnull().sum()

0

In [20]:
df["THE LAW"].isnull().sum()


0

In [21]:
df["THE FACTS"].isnull().sum()


0

In [22]:
df["PROCEDURE"].isnull().sum()


14

In [23]:
df = df[:6]

In [24]:
df = df.dropna()

In [25]:
import re
import pandas as pd

def clean_text(text, patterns_to_remove):
    # Create a regular expression pattern by joining the provided patterns with '|'
    pattern = '|'.join(re.escape(pat) for pat in patterns_to_remove)
    
    # Remove the specified patterns except single newlines '\n'
    cleaned_text = re.sub(pattern, '', text)
    cleaned_text = re.sub(r'(\d+\.\s*)', ' ', cleaned_text)  # Remove digits and dots
    cleaned_text = re.sub(r'(\n{2,})', ' ', cleaned_text)  # Replace multiple newlines with a single space
    return cleaned_text.strip()  # Strip leading/trailing spaces


# Apply the clean_text function to the 'procedure' column
df['Case Name'] = df['Case Name'].apply(lambda x: clean_text(x, ['\n\n', '\n']))
df['Committee Members'] = df['Committee Members'].apply(lambda x: clean_text(x, ['\n\n', '\n']))
df['PROCEDURE'] = df['PROCEDURE'].apply(lambda x: clean_text(x, ['\n\n', '\n']))
df['THE FACTS'] = df['THE FACTS'].apply(lambda x: clean_text(x, ['\n\n', '\n']))
df['THE LAW'] = df['THE LAW'].apply(lambda x: clean_text(x, ['\n\n', '\n']))
df['Judgment'] = df['Judgment'].apply(lambda x: clean_text(x, ['\n\n', '\n']))

# Print the cleaned DataFrame
# df.head()


In [26]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

In [27]:
df.head()

Case Name                                                                                                                                                                        Committee Members  \
0  Terge v. Hungary        Faris Vehabović, President,Carlo Ranzoni,Péter Paczolay, judges,and Andrea Tamietti, Deputy Section Registrar,                                                                            
1  Hristoskov v. Bulgaria  Gabriele Kucsko-Stadlmayer, President,Yonko Grozev,Lado Chanturia, judges,and Anne-Marie Dougin, Acting Deputy Section Registrar,                                                         
2  Pihoni v. Albania       Robert Spano, President,Paul Lemmens,Ledi Bianku,Işıl Karakaş,Nebojša Vučinić,Valeriu Griţco,Stéphanie Mourou-Vikström, judges,and Stanley Naismith, Section Registrar,                   
3  Mskhiladze v. Russia    Helen Keller, President,Pere Pastor Vilanova,Alena Poláčková, judges,and Fatoş Aracı, Deputy Section Registrar,                                                                           
4  Akimenkov v. Russia     Helena Jäderblom, President,Luis López Guerra,Dmitry Dedov,Pere Pastor Vilanova,Alena Poláčková,Georgios A. Serghides,Jolien Schukking, judges,and Stephen Phillips, Section Registrar,   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        PROCEDURE  \
0  The case originated in an application (no. 3625/15) against Hungary lodged with the Court under Article 34 of the Convention for the Protection of Human Rights and Fundamental Freedoms (“the Convention”) by a Hungarian national, Mr István Terge (“the applicant”), on 9 January   The applicant was represented by Mr D. Karsai, a lawyer practising in Budapest. The Hungarian Government (“the Government”) were represented by Mr Z. Tallódi, Agent at the Ministry of Justice. On 19 January 2017 the application was communicated to the Government.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
1  The case originated in an application (no. 50760/09) against the Republic of Bulgaria lodged with the Court under Article 34 of the Convention for the Protection of Human Rights and Fundamental Freedoms (“the Convention”) by a Bulgarian national, Mr Emil Petrov Hristoskov (“the applicant”), on 15 August   The applicant was represented by Ms S. Marg

In [28]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

def remove_starting_comma(text):
    return re.sub(r'^, ', '', text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Najeeb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
columns_to_clean = ['Case Name', 'Committee Members', 'PROCEDURE', 'THE FACTS', 'THE LAW', 'Judgment']

df[columns_to_clean] = df[columns_to_clean].applymap(lambda x: clean_text(x, ['\n\n', '\n']))
df[columns_to_clean] = df[columns_to_clean].applymap(lambda x: x.lower())
df[columns_to_clean] = df[columns_to_clean].applymap(lambda x: re.sub("'", '', x))
df['Judgment'] = df['Judgment'].apply(remove_starting_comma)
# df[columns_to_clean] = df[columns_to_clean].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
# df[columns_to_clean] = df[columns_to_clean].apply(lambda x: re.sub("'", '', x))

<font family="Arial, sans-serif" size="8px" color="#D2E0FB">CHAT GPT TEXT GENERATION</font>


In [30]:
data = df

In [31]:
# data['THE FACTS'] = data['THE FACTS'].astype(str)
# data['THE LAW'] = data['THE LAW'].astype(str)
data['THE FACTS'] = data['THE FACTS'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
data['THE LAW'] = data['THE LAW'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Combine input columns (e.g., 'PROCEDURE', 'THE FACTS', 'THE LAW') into a single text column
# data['input_text'] = data['PROCEDURE'] + ' ' + data['THE FACTS'] + ' ' + data['THE LAW']
data['input_text'] = data['THE FACTS'] + ' ' + data['THE LAW']


# Ensure all values in 'input_text' are strings
data['input_text'] = data['input_text'].astype(str)

In [32]:
df = data

In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Assuming you have a DataFrame df with 'input_text' and 'Judgment' columns
input_texts = df['input_text'].astype(str).tolist()
judgments = df['Judgment'].astype(str).tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_texts + judgments)
input_sequences = tokenizer.texts_to_sequences(input_texts)
judgment_sequences = tokenizer.texts_to_sequences(judgments)

# Pad sequences to a fixed length
max_sequence_length = 1000  # Adjust as needed
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
judgment_sequences = pad_sequences(judgment_sequences, maxlen=max_sequence_length, padding='post')

# Define the model architecture
embedding_dim = 256
hidden_units = 512

input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim)(input_layer)
lstm_layer = LSTM(hidden_units, return_sequences=True)(embedding_layer)
output_layer = Dense(len(tokenizer.word_index) + 1, activation='softmax')(lstm_layer)

model = Model(inputs=input_layer, outputs=output_layer)

optimizer = Adam(learning_rate=0.001)
loss_fn = SparseCategoricalCrossentropy()
accuracy_metric = SparseCategoricalAccuracy()

model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy_metric])

# Split data into training and validation sets
train_ratio = 0.8
split_index = int(len(input_sequences) * train_ratio)

input_train, input_val = input_sequences[:split_index], input_sequences[split_index:]
judgment_train, judgment_val = judgment_sequences[:split_index], judgment_sequences[split_index:]

# Define callbacks for training
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('legal_text_generation_model.h5', save_best_only=True)

# Train the model using teacher forcing
batch_size = 32
epochs = 1

# Prepare target sequences for teacher forcing (shifted by one time step)
judgment_sequences_shifted = np.roll(judgment_sequences, shift=-1, axis=1)
judgment_train_shifted, judgment_val_shifted = judgment_sequences_shifted[:split_index], judgment_sequences_shifted[split_index:]

model.fit(input_train, judgment_train_shifted, batch_size=batch_size, epochs=epochs,
        validation_data=(input_val, judgment_val_shifted),
        callbacks=[early_stopping, model_checkpoint])

# Now, you can use the trained model for inference on new cases, as described in the previous responses.


1/1 [==============================] - 9s 9s/step - loss: 8.3559 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 8.3379 - val_sparse_categorical_accuracy: 0.6775


In [21]:

input_text1 = df['input_text'][0]
input_sequences1 = tokenizer.texts_to_sequences(input_text1)
input_sequences1 = pad_sequences(input_sequences1, maxlen=max_sequence_length, padding='post')
predicted_sequence = model.predict(input_sequences1)

predicted_indices = np.argmax(predicted_sequence, axis=-1)
predicted_text = tokenizer.sequences_to_texts(predicted_indices)
print(predicted_text)


# Assuming you have a DataFrame df with 'input_text' column
input_text = df['input_text'][0]  # Replace [0] with the index of the text you want to use as input
# predicted_output = generate_text(input_text, model, tokenizer, max_sequence_length)
print("Input Text:")
print(input_text)
print("\nPredicted Output:")
# print(predicted_output)

  6/979 [..............................] - ETA: 1:39:41

<font family="Arial, sans-serif" size="8px" color="#D2E0FB">MACHINE TRANSLATION</font>


In [21]:
# exclude = set(string.punctuation) # Set of all special characters
# # Remove all the special characters
# lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
# lines['hindi']=lines['hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [130]:
lines = df

In [131]:
lines['Judgment'][0]

'the court, unanimously, declares the application admissible; holds that there has been no violation of the substantive limb of article\xa03 of the convention; holds that there has been a violation of the procedural limb of article 3 of the convention; holds(a)\xa0\xa0that the respondent state is to pay the applicant, within three months, the following amounts, to be converted into the currency of the respondent state at the rate applicable at the date of settlement:(i)\xa0\xa0eur 6,000 (six thousand euros), plus any tax that may be chargeable, in respect of pecuniary damage;(ii)\xa0\xa0eur 2,000 (two thousand euros), plus any tax that may be chargeable to the applicant, in respect of costs and expenses;(b)\xa0\xa0that from the expiry of the above-mentioned three months until settlement simple interest shall be payable on the above amounts at a rate equal to the marginal lending rate of the european central bank during the default period plus three percentage points; dismisses the rema

In [132]:
lines.head()

,Case Name,Committee Members,PROCEDURE,THE FACTS,THE LAW,Judgment,input_text
0,terge v. hungary,"faris vehabović, president,carlo ranzoni,péter...",the case originated in an application (no. 362...,i. the circumstances of the case the applican...,i. alleged violation of article 3 of the conv...,"the court, unanimously, declares the applicati...",i. the circumstances of the case the applican...
1,hristoskov v. bulgaria,"gabriele kucsko-stadlmayer, president,yonko gr...",the case originated in an application (no. 507...,i. the circumstances of the case the applican...,i. scope of the case after notice of the appl...,"the court, unanimously, declares the complaint...",i. the circumstances of the case the applican...
2,pihoni v. albania,"robert spano, president,paul lemmens,ledi bian...",the case originated in an application (no. 743...,i. the circumstances of the case the applican...,i. alleged violation of articles 3 and 13 of ...,the court unanimously decides to join to the m...,i. the circumstances of the case the applican...
3,mskhiladze v. russia,"helen keller, president,pere pastor vilanova,a...",the case originated in an application (no. 477...,i. the circumstances of the case the applican...,i. alleged violation of article 3 of the conv...,"the court, unanimously, declares the complaint...",i. the circumstances of the case the applican...
4,akimenkov v. russia,"helena jäderblom, president,luis lópez guerra,...",the case originated in two applications (nos. ...,i. the circumstances of the case the applican...,i. joinder of the applications given their co...,"the court, unanimously, decides to join the ap...",i. the circumstances of the case the applican...


In [133]:
# Add start and end tokens to target sequences
lines['Judgment'] = lines['Judgment'].apply(lambda x : 'START_ '+ x + ' _END')

In [134]:
lines['THE FACTS'] = lines['THE FACTS'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
lines['THE LAW'] = lines['THE LAW'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Combine input columns (e.g., 'PROCEDURE', 'THE FACTS', 'THE LAW') into a single text column
# lines['input_text'] = lines['PROCEDURE'] + ' ' + lines['THE FACTS'] + ' ' + lines['THE LAW']
lines['input_text'] = lines['THE FACTS'] + ' ' + lines['THE LAW']

# Ensure all values in 'input_text' are strings
lines['input_text'] = lines['input_text'].astype(str)


In [135]:
### Get English and Hindi Vocabulary
all_input_words=set()
for eng in lines['input_text']:
    for word in eng.split():
        if word not in all_input_words:
            all_input_words.add(word)

all_judgment_words=set()
for hin in lines['Judgment']:
    for word in hin.split():
        if word not in all_judgment_words:
            all_judgment_words.add(word)

In [143]:
lines.head()

,Case Name,Committee Members,PROCEDURE,THE FACTS,THE LAW,Judgment,input_text,length_input,length_judgment
0,terge v. hungary,"faris vehabović, president,carlo ranzoni,péter...",the case originated in an application (no. 362...,i. the circumstances of the case the applican...,i. alleged violation of article 3 of the conv...,"START_ the court, unanimously, declares the ap...",i. the circumstances of the case the applican...,4960,163
1,hristoskov v. bulgaria,"gabriele kucsko-stadlmayer, president,yonko gr...",the case originated in an application (no. 507...,i. the circumstances of the case the applican...,i. scope of the case after notice of the appl...,"START_ the court, unanimously, declares the co...",i. the circumstances of the case the applican...,2074,221
3,mskhiladze v. russia,"helen keller, president,pere pastor vilanova,a...",the case originated in an application (no. 477...,i. the circumstances of the case the applican...,i. alleged violation of article 3 of the conv...,"START_ the court, unanimously, declares the co...",i. the circumstances of the case the applican...,4446,246
7,a. v. switzerland,"helena jäderblom, president,branko lubarda,hel...",the case originated in an application (no. 603...,i. the circumstances of the case the applican...,i. alleged violation of articles 2 and 3 of t...,"START_ the court, unanimously, declares the ap...",i. the circumstances of the case the applican...,4558,79
20,á.r. v. hungary,"faris vehabović, president,carlo ranzoni,péter...",the case originated in an application (no. 204...,i. the circumstances of the case the applican...,i. alleged violation of article 8 of the conv...,"START_ the court, unanimously, declares the co...",i. the circumstances of the case the applican...,1366,188


## continue from here

In [137]:
lines['length_input']=lines['input_text'].apply(lambda x:len(x.split(" ")))
lines['length_judgment']=lines['Judgment'].apply(lambda x:len(x.split(" ")))

In [138]:
lines[lines['length_judgment']<400].shape

(38, 9)

In [139]:
lines.head()
lines[lines['length_input']>1000].shape

(39, 9)

In [140]:
lines['input_text'].head()

0    i.  the circumstances of the case the applican...
1    i.  the circumstances of the case the applican...
2    i.  the circumstances of the case the applican...
3    i.  the circumstances of the case the applican...
4    i.  the circumstances of the case the applican...
Name: input_text, dtype: object

In [141]:
lines['length_input']

0      4960
1      2074
2      9910
3      4446
4     10197
5      8619
6      8116
7      4558
8     11888
9     14046
10    11543
11     8550
12     6292
13     7830
14     5605
15    13204
16    11483
17     7572
18     5780
19     5621
20     1366
21     4377
22     5182
23     3994
24     8137
25     8382
26     6888
28    21037
29     4857
30     4889
31     4068
32    21739
33    13898
34     5956
35    16370
36     6242
37     6667
38    10819
39    11051
Name: length_input, dtype: int64

In [142]:
lines=lines[lines['length_input']<=5000]
lines=lines[lines['length_judgment']<=500]

In [144]:
print("maximum length of Hindi Sentence ",max(lines['length_judgment']))
print("maximum length of English Sentence ",max(lines['length_input']))

maximum length of Hindi Sentence  365
maximum length of English Sentence  4960


In [145]:
max_length_src=max(lines['length_judgment'])
max_length_tar=max(lines['length_input'])

In [146]:
input_words = sorted(list(all_input_words))
target_words = sorted(list(all_judgment_words))
num_encoder_tokens = len(all_input_words)
num_decoder_tokens = len(all_judgment_words)
num_encoder_tokens, num_decoder_tokens

(20112, 567)

In [147]:
num_decoder_tokens

567

In [148]:
num_decoder_tokens += 1

In [149]:
num_decoder_tokens

568

In [150]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [151]:
input_token_index

{'&': 1,
 '(': 2,
 '((dec.)': 3,
 '((dec.),': 4,
 '(...)': 5,
 '(...).prisoners': 6,
 '(...).“in': 7,
 '(...):“....': 8,
 '(1': 9,
 '(1)': 10,
 '(16': 11,
 '(16)(1)(b)(iii)': 12,
 '(17': 13,
 '(17/01/2012),': 14,
 '(18': 15,
 '(1998)': 16,
 '(1999)': 17,
 '(2': 18,
 '(2%': 19,
 '(2)': 20,
 '(2),': 21,
 '(2004)': 22,
 '(2006)2': 23,
 '(2007)': 24,
 '(2010)': 25,
 '(2011)': 26,
 '(2012)': 27,
 '(2012a,': 28,
 '(2015)': 29,
 '(2016)': 30,
 '(2017)': 31,
 '(27': 32,
 '(3)': 33,
 '(365': 34,
 '(365cm': 35,
 '(4)': 36,
 '(5': 37,
 '(5)': 38,
 '(5)).': 39,
 '(552,195': 40,
 '(6)': 41,
 '(6),': 42,
 '(7)': 43,
 '(9': 44,
 '([gc]': 45,
 '([gc],': 46,
 '([that': 47,
 '(a': 48,
 '(a)': 49,
 '(a)(4)': 50,
 '(a),': 51,
 '(a).': 52,
 '(a.s.)': 53,
 '(a/hrc/29/crp.1).': 54,
 '(a/hrc/31/69),': 55,
 '(a/hrc/32/47),': 56,
 '(a/hrc/32/crp.1).b.': 57,
 '(a/hrc/34/65,': 58,
 '(a/res/40/33),': 59,
 '(abolished': 60,
 '(about': 61,
 '(abuse': 62,
 '(according': 63,
 '(accounts': 64,
 '(accreditati)': 65,
 '(

In [152]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [153]:
reverse_input_char_index

{1: '&',
 2: '(',
 3: '((dec.)',
 4: '((dec.),',
 5: '(...)',
 6: '(...).prisoners',
 7: '(...).“in',
 8: '(...):“....',
 9: '(1',
 10: '(1)',
 11: '(16',
 12: '(16)(1)(b)(iii)',
 13: '(17',
 14: '(17/01/2012),',
 15: '(18',
 16: '(1998)',
 17: '(1999)',
 18: '(2',
 19: '(2%',
 20: '(2)',
 21: '(2),',
 22: '(2004)',
 23: '(2006)2',
 24: '(2007)',
 25: '(2010)',
 26: '(2011)',
 27: '(2012)',
 28: '(2012a,',
 29: '(2015)',
 30: '(2016)',
 31: '(2017)',
 32: '(27',
 33: '(3)',
 34: '(365',
 35: '(365cm',
 36: '(4)',
 37: '(5',
 38: '(5)',
 39: '(5)).',
 40: '(552,195',
 41: '(6)',
 42: '(6),',
 43: '(7)',
 44: '(9',
 45: '([gc]',
 46: '([gc],',
 47: '([that',
 48: '(a',
 49: '(a)',
 50: '(a)(4)',
 51: '(a),',
 52: '(a).',
 53: '(a.s.)',
 54: '(a/hrc/29/crp.1).',
 55: '(a/hrc/31/69),',
 56: '(a/hrc/32/47),',
 57: '(a/hrc/32/crp.1).b.',
 58: '(a/hrc/34/65,',
 59: '(a/res/40/33),',
 60: '(abolished',
 61: '(about',
 62: '(abuse',
 63: '(according',
 64: '(accounts',
 65: '(accreditati)',
 66

In [154]:
lines.head(10)

,Case Name,Committee Members,PROCEDURE,THE FACTS,THE LAW,Judgment,input_text,length_input,length_judgment
0,terge v. hungary,"faris vehabović, president,carlo ranzoni,péter...",the case originated in an application (no. 362...,i. the circumstances of the case the applican...,i. alleged violation of article 3 of the conv...,"START_ the court, unanimously, declares the ap...",i. the circumstances of the case the applican...,4960,163
1,hristoskov v. bulgaria,"gabriele kucsko-stadlmayer, president,yonko gr...",the case originated in an application (no. 507...,i. the circumstances of the case the applican...,i. scope of the case after notice of the appl...,"START_ the court, unanimously, declares the co...",i. the circumstances of the case the applican...,2074,221
3,mskhiladze v. russia,"helen keller, president,pere pastor vilanova,a...",the case originated in an application (no. 477...,i. the circumstances of the case the applican...,i. alleged violation of article 3 of the conv...,"START_ the court, unanimously, declares the co...",i. the circumstances of the case the applican...,4446,246
7,a. v. switzerland,"helena jäderblom, president,branko lubarda,hel...",the case originated in an application (no. 603...,i. the circumstances of the case the applican...,i. alleged violation of articles 2 and 3 of t...,"START_ the court, unanimously, declares the ap...",i. the circumstances of the case the applican...,4558,79
20,á.r. v. hungary,"faris vehabović, president,carlo ranzoni,péter...",the case originated in an application (no. 204...,i. the circumstances of the case the applican...,i. alleged violation of article 8 of the conv...,"START_ the court, unanimously, declares the co...",i. the circumstances of the case the applican...,1366,188
21,amirov v. russia,"luis lópez guerra, president,dmitry dedov,joli...",the case originated in an application (no. 562...,i. the circumstances of the case the applican...,i. alleged violation of article 3 of the conv...,"START_ the court, unanimously, declares the co...",i. the circumstances of the case the applican...,4377,238
23,mitev v. bulgaria,"erik møse, president,yonko grozev,gabriele kuc...",the case originated in an application (no. 341...,i. the circumstances of the case the applican...,alleged violation of articles 3 and 13 of the ...,"START_ the court, unanimously, declares the ap...",i. the circumstances of the case the applican...,3994,49
29,dejnek v. poland,"linos-alexandre sicilianos, president,kristina...",the case originated in an application (no. 963...,i. the circumstances of the case the applican...,i. scope of the application the court first n...,"START_ the court, unanimously, declares the co...",i. the circumstances of the case the applican...,4857,168
30,apcov v. the republic of moldova and russia,"işıl karakaş, president,julia laffranque,paul ...",the case originated in an application (no. 134...,i. the circumstances of the case the applican...,the applicant submitted that he had been arres...,"START_ the court declares, unanimously, the co...",i. the circumstances of the case the applican...,4889,365
31,golubar v. croatia,"işıl karakaş, president,julia laffranque,paul ...",the case originated in an application (no. 219...,i. the circumstances of the case the applican...,i. alleged violation of article 3 the convent...,"START_ the court, unanimously, declares the co...",i. the circumstances of the case the applican...,4068,41


In [155]:
from sklearn.model_selection import train_test_split
X, y = lines['input_text'], lines['Judgment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((8,), (2,))

In [156]:
X_train

21    i.  the circumstances of the case the applican...
0     i.  the circumstances of the case the applican...
29    i.  the circumstances of the case the applican...
3     i.  the circumstances of the case the applican...
31    i.  the circumstances of the case the applican...
20    i.  the circumstances of the case the applican...
7     i.  the circumstances of the case the applican...
23    i.  the circumstances of the case the applican...
Name: input_text, dtype: object

In [157]:
encoder_input_data = np.zeros((2, max_length_src),dtype='float32')
decoder_input_data = np.zeros((2, max_length_tar),dtype='float32')
decoder_target_data = np.zeros((2, max_length_tar, num_decoder_tokens),dtype='float32')

In [158]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [159]:
latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [160]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens+1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [161]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [162]:
model.summary()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 10

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, None, 300)            6033900   ['input_2[0][0]']             
                                                                                                  
 embedding_2 (Embedding)     (None, None, 300)            170700    ['input_3[0][0]']             
                                                                                            

In [163]:
model.save('eng-to-hindi.h5')

/home/student/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [164]:
a, b = next(generate_batch())

IndexError: index 365 is out of bounds for axis 1 with size 365

In [ ]:
b

In [165]:
X_train[4]

KeyError: 4

In [166]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=10,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

/tmp/ipykernel_1175/1093200829.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


IndexError: index 365 is out of bounds for axis 1 with size 365

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
ref = [
    'this is moonlight'.split(),
    'Look, this is moonlight'.split(),
    'moonlight it is'.split()
]
test = 'it is moonlight'.split()
print('BLEU score for test-> {}'.format(sentence_bleu(ref, test)))
 
test01 = 'it is cat and moonlight'.split()
print('BLEU score for test01-> {}'.format(sentence_bleu(ref, test01)))